# Part 7 - Federated Learning with FederatedDataset

Here we introduce a new tool for using federated datasets. We have created a `FederatedDataset` class which is intended to be used like the PyTorch Dataset class, and is given to a federated data loader `FederatedDataLoader` which will iterate on it in a federated fashion.


Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

We use the sandbox that we discovered last lesson

In [1]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), verbose=False)

/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/atrask/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Setting up Sandbox...
Done!


Then search for a dataset

In [9]:
boston_data = grid.search("#boston", "#data", verbose=False, return_counter=False)
boston_target = grid.search("#boston", "#target", verbose=False, return_counter=False)

We load a model and an optimizer

In [10]:
n_features = boston_data['alice'][0].shape[1]
n_targets = 1
model = th.nn.Linear(n_features, n_targets)
optimizer = th.optim.SGD(params=model.parameters(),lr=0.0000001)

Here we cast the data fetched in a `FederatedDataset`. See the workers which hold part of the data.

In [11]:
dataset = sy.FederatedDataset(boston_data, boston_target)
print(dataset.workers)

['jon', 'alice', 'theo', 'jason', 'bob', 'andy']


We put it in a `FederatedDataLoader` and specify options

In [12]:
train_loader = sy.FederatedDataLoader(dataset, batch_size=4, shuffle=False, drop_last=False)

And finally we iterate over epochs. You can see how similar this is compared to pure and local PyTorch training!

In [13]:
epochs = 10
for epoch in range(1, epochs + 1):
    loss_accum = 0
    for batch_idx, (data, target) in enumerate(train_loader):

        model.send(data.location)
        
        optimizer.zero_grad()
        pred = model(data)
        loss = ((pred - target)**2).sum()
        loss.backward()
        optimizer.step()
        
        model.get()
        loss = loss.get()
        
        loss_accum += float(loss)
        
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * data.shape[0], len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
            
            
    print('Total loss', loss_accum)


Train Epoch: 1 [0/506 (0%)]	Loss: 257324.187500
Train Epoch: 1 [80/506 (16%)]	Loss: 164.807587
Train Epoch: 1 [80/506 (31%)]	Loss: 50.742851
Train Epoch: 1 [120/506 (47%)]	Loss: 265.615936
Train Epoch: 1 [160/506 (63%)]	Loss: 977.800781
Train Epoch: 1 [200/506 (79%)]	Loss: 37.221901
Train Epoch: 1 [240/506 (94%)]	Loss: 36.468460
Total loss 650352.809401989
Train Epoch: 2 [0/506 (0%)]	Loss: 409.260925
Train Epoch: 2 [40/506 (16%)]	Loss: 57.283073
Train Epoch: 2 [80/506 (31%)]	Loss: 39.606285
Train Epoch: 2 [120/506 (47%)]	Loss: 164.271698
Train Epoch: 2 [160/506 (63%)]	Loss: 941.646362
Train Epoch: 2 [200/506 (79%)]	Loss: 30.483576
Train Epoch: 2 [240/506 (94%)]	Loss: 29.300154
Total loss 49260.60708284378
Train Epoch: 3 [0/506 (0%)]	Loss: 195.616425
Train Epoch: 3 [40/506 (16%)]	Loss: 127.807770
Train Epoch: 3 [80/506 (31%)]	Loss: 31.930153
Train Epoch: 3 [120/506 (47%)]	Loss: 121.225540
Train Epoch: 3 [160/506 (63%)]	Loss: 915.360107
Train Epoch: 3 [200/506 (79%)]	Loss: 27.724627
Trai

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)